In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext line_profiler


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from snmfem.experiments import load_samples, load_data, run_experiment

In [ ]:
# dataset = "dataset_EDXS_small.json"
dataset = "dataset_EDXS.json"
samples, k, shape_2d = load_samples(dataset)
n_samples = 5
samples = samples[:n_samples]


In [ ]:
default_params = {
    "n_components" : k,
    "tol" : 1e-6,
    "max_iter" : 10,
    "init" : "random",
    "random_state" : 1,
    "verbose" : 0
    }

params_snmf = {
    "force_simplex" : True,
    "skip_G" : False,
    "mu": 0
}

params_evalution = {
    "u" : True,
}

exp = {"name": "snmfem smooth 100", "method": "SmoothNMF", "params": {**default_params, **params_snmf, "lambda_L" : 100.0, "shape_2d" : shape_2d}}

In [ ]:
sample_num = 0

s = samples[sample_num]
Xflat, true_spectra, true_maps, G = load_data(s)

In [ ]:
from snmfem.updates import multiplicative_step_a, multiplicative_step_p
from snmfem.estimators import SmoothNMF, NMFEstimator
from snmfem.measures import KLdiv

In [ ]:
%lprun -f NMFEstimator.fit_transform run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution)

In [ ]:
%lprun -f SmoothNMF._iteration run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution)

In [ ]:
%lprun -f multiplicative_step_a run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution)

In [ ]:
%lprun -f multiplicative_step_p run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution)

In [ ]:
%lprun -f SmoothNMF.loss run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution)

In [ ]:
%lprun -f NMFEstimator.loss run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution)

In [ ]:
%lprun -f KLdiv run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution)

In [ ]:
# %timeit -r 1 -n 1 run_experiment(Xflat, true_spectra, true_maps, G, exp, params_evalution)

In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
A = true_maps
GP = np.linalg.lstsq(A.T, Xflat.T)[0]
P = np.linalg.lstsq(G, GP.T)[0]
X = Xflat

In [ ]:
%timeit G @ P @ A
%timeit (G @ P) @ A
%timeit G @ (P @ A)

In [ ]:
GPA = G @ P @ A
%timeit X / GPA
%timeit np.log(X+0.01)


In [ ]:
GPA = G @ P @ A
%timeit X / GPA

In [ ]:
t_G = Variable(torch.tensor(G), requires_grad=False)
t_P = Variable(torch.tensor(P), requires_grad=False)
t_A = Variable(torch.tensor(A), requires_grad=False)
t_X = Variable(torch.tensor(X), requires_grad=False)

In [ ]:
%timeit (G @ P) @ A
%timeit t_G.matmul(t_P).matmul(t_A)

In [ ]:
t_GPA = t_G.matmul(t_P).matmul(t_A)
%timeit t_X / t_GPA